In [1]:
import ROOT

OBJ: TStyle	ildStyle	ILD Style : 0 at: 0x41a7e90
Welcome to JupyROOT 6.28/10


In [2]:
%jsroot on

In [3]:
ROOT.ildStyle.SetOptStat(1)

In [4]:
%%cpp
using namespace ROOT::VecOps;

In [5]:
df = ROOT.RDataFrame("events", ("data/truejet/test/sw_sl/eLpL_truejet.edm4hep.root", "data/truejet/test/sw_sl/eLpR_truejet.edm4hep.root"))
# df = ROOT.RDataFrame("events", "data/truejet/test/sw_sl/eLpL_truejet.edm4hep.root")
# df = ROOT.RDataFrame("events", "data/truejet/test/sw_sl/eLpR_truejet.edm4hep.root")

In [6]:
%%cpp
//auto bPol_e = [](unsigned int slot, const ROOT::RDF::RSampleInfo &id) { return id.AsString(); };
auto bPol_e = [](unsigned int slot, const ROOT::RDF::RSampleInfo &id) { return id.Contains("eL") ? -1 : 1; };
auto bPol_p = [](unsigned int slot, const ROOT::RDF::RSampleInfo &id) { return id.Contains("pL") ? -1 : 1; };

In [7]:
df = df.DefinePerSample("beamPol_e", "bPol_e(rdfslot_, rdfsampleinfo_)")
df = df.DefinePerSample("beamPol_p", "bPol_p(rdfslot_, rdfsampleinfo_)")

In [8]:
# let's start with the InitialColourNeutrals as they have set energy and momentum (and mass, I hope it's consistent ;))

# rely on the relevant particle ID being the first one
# returns a vector containing the idx of the first PID in the ICN_PID collection belonging to the ICN
df = df.Define("pid_idx", "InitialColourNeutrals.particleIDs_begin")

# get the masks for the qq and lnu InitialColourNeutrals (W's)
# for each pid idx take the ICN_PID type and check if it is 1 (quarks) or 2 (lepton)
# returns a vector of ICN.size() with a 1 at the corresponding position, can be used to access all ICN.something fields
df = df.Define("qq_ICN_mask", "Take(InitialColourNeutrals_particleIDs.type, pid_idx) == 1")
df = df.Define("lnu_ICN_mask", "Take(InitialColourNeutrals_particleIDs.type, pid_idx) == 2")

# get the masses of the InitialColourNeutrals (inv. mass of the lnu and qqbar systems)
df = df.Define("m_qq", "InitialColourNeutrals.mass[qq_ICN_mask]")
df = df.Define("m_lnu", "InitialColourNeutrals.mass[lnu_ICN_mask]")

# should be able to get e, nu, q, qbar + gluon jets by looking at which truejets belong to the INC
# I only need the separate q when I also try to do quark charge stuff so for now I am happy with qq ICN
# returns a vector of size two with the indices of l and nu in the truejet collection
df = df.Define("lnu_TJ_idx", "Range(InitialColourNeutrals.particles_begin[lnu_ICN_mask], InitialColourNeutrals.particles_begin[lnu_ICN_mask])")

# TrueJets and TrueJets_particleIDs have the same number of entries and in the same order (this assumption should be safe)
# Take the TJ_PID.PDGs and check if e or nu
# returns vector of TrueJets.size() with a 1 where the l is
# FIXME: need to change hardcoded comparison for other l than e
df = df.Define("l_TJ_mask", "abs(Take(TrueJets_particleIDs.PDG, lnu_TJ_idx)) == 11")
df = df.Define("nu_TJ_mask", "abs(Take(TrueJets_particleIDs.PDG, lnu_TJ_idx)) == 12")

# As we only deal with exactly _one_ l and _one_ nu it might be easier to use the idx instead of the mask as
# RVec[mask] -> RVec, but RVec[idx] -> single element
# so that we don't need to worry about the additional vector layer
# the idx is the only 1 in the mask, the rest is 0 -> just use ArgMax
df = df.Define("l_TJ_idx", "ArgMax(l_TJ_mask)")
df = df.Define("nu_TJ_idx", "ArgMax(nu_TJ_mask)")

# FIXME: for debugging
# df = df.Define("multi_qq", "Sum(qq_mask) > 1")

In [9]:
h_m_qq = df.Histo1D("m_qq")
h_m_lnu = df.Histo1D("m_lnu")

h_m_lnu_eLpL = df.Filter("beamPol_e == -1 && beamPol_p == -1").Histo1D("m_lnu")

In [10]:
c_m_qq = ROOT.TCanvas()
h_m_qq.Draw()
c_m_qq.Draw()

c_m_lnu = ROOT.TCanvas()
h_m_lnu.Draw()
c_m_lnu.Draw()

c_m_lnu_eLpL = ROOT.TCanvas()
h_m_lnu_eLpL.Draw()
c_m_lnu_eLpL.Draw()

In [11]:
# FIXME: for debug
# multi_qq_evts = df.Filter("multi_qq").Take["ULong64_t"]("rdfentry_").GetValue()
# print(multi_qq_evts)